In [ ]:
import pandas as pd
import requests
from tqdm.notebook import tqdm
import os
import time
pd.set_option("display.max_columns", None)

In [ ]:
df =  pd.read_csv('xc_metadata_curated.csv')
df['path'] = df.apply(lambda row: os.path.join(os.path.abspath('data'), row.gensp, str(row.id) + '.' + row.filename.split('.')[-1].lower()), axis=1).values

In [ ]:
for row in tqdm(df.itertuples()):
    specie_dir = os.path.join('data', row.gensp)
    os.makedirs(specie_dir, exist_ok=True)
    time.sleep(1.) # no matter what, don't mess with this sleeping time, your ip address might get blocked if you download fast and a lot from Xeno-Canto o_O !!
    file_format = row.filename.rsplit('.', 1)[1].lower()  # e.g, [filename, Mp3][1].lower() -> mp3
    file_name = str(row.id) + '.' + file_format  # e.g., 638123.mp3
    path_to_save = os.path.join(specie_dir, file_name)
    if not os.path.isfile(path_to_save):  # do not re-download if the file exists
        try:
            response = requests.get(row.file, timeout=30, allow_redirects=True)
            if response.status_code == 200 and response.content:
                with open(path_to_save, 'wb') as f:
                    f.write(response.content)
            else:
                print(f'failed {row.id} with link {row.file}')
        except Exception as e:
            print(f'failed to download {row.file}!')

In [ ]:
for i, f in tqdm(enumerate(df['path'].values)):
    if not os.path.isfile(f):
        print(f'failed to download record at index {i}: {f}')
    break